In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

True 2.1.1+cu118


In [1]:
from src.api import Trainer  
app = Trainer.initialize(stage = 0 , resume = 1 , checkname= 1)
app.go()

Use device name: NVIDIA GeForce RTX 4090


24-06-30 23:13:53|MOD:display     |: Model Specifics:
24-06-30 23:13:53|MOD:display     |: Start Process [Data] at Sun Jun 30 23:13:53 2024!


--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to ple_gru_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=10,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
Callback : DetailedAlphaAnalysis() , record and concat each model to Alpha model instance
{'random_seed': None,
 'model_name': 'ple

24-06-30 23:13:55|MOD:display     |: Finish Process [Data], Cost 2.3 Secs
24-06-30 23:13:55|MOD:display     |: Start Process [Fit] at Sun Jun 30 23:13:55 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
First Iterance: (20240604, 0)
score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


FirstBite Ep#  0 : loss  3.99247, train 0.00559, valid 0.01930, best 0.0193, lr1.3e-03
FirstBite Ep#  5 : loss  2.97126, train 0.09069, valid 0.10078, best 0.1008, lr2.5e-03
FirstBite Ep# 10 : loss  2.92364, train 0.10848, valid 0.10585, best 0.1058, lr1.9e-03
FirstBite Ep# 15 : loss  2.87046, train 0.12774, valid 0.09597, best 0.1114, lr1.0e-07
FirstBite Ep# 20 : loss  2.85631, train 0.12936, valid 0.09943, best 0.1114, lr9.4e-04
FirstBite Ep# 25 : loss  2.81432, train 0.14227, valid 0.10842, best 0.1114, lr3.1e-04
FirstBite Ep# 30 : loss  2.80182, train 0.14562, valid 0.10753, best 0.1120, lr1.6e-04


In [5]:
from src import api

In [29]:
api.mod.net.state_dict()['layers.0.shared_expert.expert.weight_ih_l0'].grad

In [34]:
list(api.mod.net.named_parameters())[1][1].grad

tensor([[nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan],


In [35]:
api.mod.metrics.loss.backward(retain_graph=True)
api.mod.optimizer.optimizer.zero_grad()
for name , param in api.mod.net.named_parameters():
    if param.grad is not None and torch.isnan(param.grad).any():
        print(name , param , param.grad)
        break

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
from src.api import DataAPI , Trainer
DataAPI.reconstruct_train_data()
Trainer.update_models()

predict is False , Data Processing start!
4 datas :['y', 'day', '30m', 'risk']
y blocks loading start!
 --> labels blocks reading [ret10_lag] DataBase...... finished! Cost 22.28 secs
 --> labels blocks reading [ret20_lag] DataBase...... finished! Cost 21.73 secs
 --> labels blocks merging (2)...... finished! Cost 4.75 secs
 --> models blocks reading [risk_exp] DataBase...... finished! Cost 114.89 secs
y blocks loading finished! Cost 269.22 secs
y blocks process...... finished! Cost 128.26 secs
y blocks masking...... finished! Cost 1.38 secs
y blocks saving ...... finished! Cost 2.88 secs
y blocks norming...... finished! Cost 0.00 secs
y finished! Cost 402.61 Seconds
--------------------------------------------------------------------------------
day blocks loading start!
 --> trade blocks reading [day] DataBase...... finished! Cost 33.11 secs
day blocks loading finished! Cost 33.57 secs
day blocks process...... finished! Cost 5.12 secs
day blocks masking...... finished! Cost 1.16 secs


In [ ]:
from src.api import Trainer
Trainer.update_models()

Use device name: NVIDIA GeForce RTX 4090


24-06-24 01:10:43|MOD:display     |: Model Specifics:
24-06-24 01:10:43|MOD:display     |: Start Process [Data] at Mon Jun 24 01:10:43 2024!


--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gru_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=5,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gruRES_day',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_types': ['best', 'swalast', '

24-06-24 01:10:51|MOD:display     |: Finish Process [Data], Cost 7.9 Secs
24-06-24 01:10:51|MOD:display     |: Start Process [Fit] at Mon Jun 24 01:10:51 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


...

FirstBite Ep#  0 : loss  1.00382, train-0.00516, valid-0.03649, best-0.0365, lr1.3e-03
FirstBite Ep#  5 : loss  0.86311, train 0.13764, valid 0.09228, best 0.0923, lr2.5e-03
FirstBite Ep# 10 : loss  0.84547, train 0.15544, valid 0.09630, best 0.0963, lr1.9e-03
FirstBite Ep# 15 : loss  0.83038, train 0.17066, valid 0.08826, best 0.0963, lr1.0e-07
FirstBite Ep# 20 : loss  0.82732, train 0.17363, valid 0.08575, best 0.0963, lr9.4e-04
FirstBite Ep# 25 : loss  0.81729, train 0.18370, valid 0.08407, best 0.0963, lr3.1e-04
FirstBite Ep# 30 : loss  0.81421, train 0.18647, valid 0.08262, best 0.0963, lr1.6e-04
24-06-24 01:27:46|MOD:display     |: gruRTN_day #1 @20231201|FirstBite Ep# 31 EarlyStop|Train 0.1865 Valid 0.0826 BestVal 0.0826|Cost  1.9Min,  3.6Sec/Ep
FirstBite Ep#  0 : loss  1.00293, train-0.00341, valid-0.00344, best-0.0034, lr1.3e-03
FirstBite Ep#  5 : loss  0.87709, train 0.12551, valid 0.08796, best 0.1129, lr2.5e-03
FirstBite Ep# 10 : loss  0.86180, train 0.13731, valid 0.10436,

--Process Queue : Data + Fit + Test
--Confirm Resume Training!
--Model_name is set to gruRES_day!
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=5,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gruRES_day',
 'model_module': 'gru',
 'model_data_type': 'day',
 'model_types': ['best', 'swalast'

24-06-24 01:36:34|MOD:display     |: Finish Process [Data], Cost 2.0 Secs
24-06-24 01:36:34|MOD:display     |: Start Process [Fit] at Mon Jun 24 01:36:34 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}


FirstBite Ep#  0 : loss  1.01252, train-0.01521, valid-0.03742, best-0.0374, lr1.3e-03
FirstBite Ep#  5 : loss  0.91026, train 0.08135, valid 0.03185, best 0.0599, lr2.5e-03
FirstBite Ep# 10 : loss  0.89291, train 0.09436, valid 0.04522, best 0.0599, lr1.9e-03
FirstBite Ep# 15 : loss  0.87755, train 0.10648, valid 0.03959, best 0.0599, lr1.0e-07
FirstBite Ep# 20 : loss  0.87426, train 0.10831, valid 0.03543, best 0.0599, lr9.4e-04
FirstBite Ep# 21 : loss  0.87082, train 0.11088, valid 0.03980, best 0.0599, lr6.3e-04, Next attempt goes!
Retrain#1 Ep#  0 : loss  0.99637, train 0.00462, valid 0.03106, best 0.0311, lr1.3e-04
Retrain#1 Ep#  5 : loss  0.94196, train 0.05698, valid 0.03578, best 0.0560, lr2.5e-04
Retrain#1 Ep# 10 : loss  0.92553, train 0.06750, valid 0.03080, best 0.0560, lr1.9e-04
Retrain#1 Ep# 15 : loss  0.91653, train 0.07499, valid 0.03537, best 0.0560, lr1.0e-07
Retrain#1 Ep# 20 : loss  0.91509, train 0.07561, valid 0.04185, best 0.0560, lr9.4e-05
Retrain#1 Ep# 21 : loss